<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/what_is_lsdtopotools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is lsdtopotools?

Last updated by Simon M Mudd on 11/05/2023

`lsdtopotools` is a collection of topographic processing routines. It has been developed and used for scientific research; you can cite the software via its Zenodo record, which has a doi: https://zenodo.org/record/7892465#.ZFziJBHMK70

You can install `lsdtopotools` using the package management software `conda` or `mamba`. To run a topographic analysis, you call one of the `lsdtopotools` programs. You can run these from a command line but we also include a python wrapper. 

We also have `lsdviztools`, a python package used for plotting `lsdtopotools` output. `lsdviztools` contains the python wrappers for `lsdtopotools`. The objective of `lsdviztools` is to produce pretty plots that you can use in presentations and publications, it is not meant to be a general geospatial visualisation package.  

In this notebook I will install `lsdtopotools` in google colab and give some information about file formats. You can also run this notebook using the docker container for lsdtopotools: https://hub.docker.com/r/lsdtopotools/lsdtt_pytools_docker If you don't know what docker is, just ignore that part and start with the colab instructions. 

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **Download some data** section. 

First we install `lsdviztools`. This will take around a minute. It is important you do this before the `condacolab` step. 

In [ ]:
!pip install lsdviztools &> /dev/null

Now we need to install lsdtopotools. We do this using something called `mamba`. To get `mamba` we install something called `condacolab`. 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Alternatively we can do this by downloading the mamba installer directly, but this frequently leads to various coding conflicts becasue you need to keep the installer URL up to date. `condacolab` does all that for you so you don't need to worry about it. 

In [ ]:
#%%bash
#MINICONDA_INSTALLER_SCRIPT=Mambaforge-Linux-x86_64.sh
#MINICONDA_PREFIX=/usr/local
#wget https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh &> /dev/null
#chmod +x $MINICONDA_INSTALLER_SCRIPT
#./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX &> /dev/null

Now use mamba to install `lsdtopotools`. 
This step takes a bit over a minute. 

In [ ]:
!mamba install -y lsdtopotools &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.8.

In [ ]:
!lsdtt-basic-metrics -v